In [53]:
import subprocess

# 设置路径
input_video = "end.mp4"
output_video = "output.mp4"
font_path = "1.ttf"
text = "Wave\nChaser"  # 多行文字需要双反斜杠

# 构造 drawtext 表达式
drawtext1 = (
    f"drawtext=fontfile={font_path}:"
    f"text='{text}':"
    f"enable='lte(t,2)':"
    f"fontsize=80:"
    f"fontcolor=white:"
    f"x='(w-text_w)/2 + ((w-text_w-10)-(w-text_w)/2)*(t/2)':"
    f"y='(h-text_h)/2 + ((h-text_h-10)-(h-text_h)/2)*(t/2)'"
)

drawtext2 = (
    f"drawtext=fontfile={font_path}:"
    f"text='{text}':"
    f"enable='gte(t,0)':"
    f"fontsize=50:"  # 缩小后的大小
    f"fontcolor=white:"
    f"x=w-text_w-10:"
    f"y=h-text_h-10"
)

# 合并滤镜
#vf_filter = f"{drawtext1},{drawtext2}"
vf_filter = f"{drawtext2}"

# 构建 ffmpeg 命令
cmd = [
    "ffmpeg",
    "-y",
    "-i", input_video,
    "-vf", vf_filter,
    "-codec:a", "copy",
    output_video
]

# 打印命令（调试用）
print("Running command:\n", " ".join(cmd))

# 执行
try:
    subprocess.run(cmd, check=True)
    print("✅ 处理完成，文字将显示至视频结尾")
except subprocess.CalledProcessError as e:
    print("❌ 出错:", e)


Running command:
 ffmpeg -y -i end.mp4 -vf drawtext=fontfile=1.ttf:text='Wave
Chaser':enable='gte(t,0)':fontsize=50:fontcolor=white:x=w-text_w-10:y=h-text_h-10 -codec:a copy output.mp4
✅ 处理完成，文字将显示至视频结尾


In [44]:
import subprocess
import os

# 视频文件路径
intro = "open.mp4"
main = "main.mp4"
outro = "end.mp4"
output = "final_output.mp4"

# 步骤 1：创建一个文件列表给 ffmpeg
with open("file_list.txt", "w", encoding="utf-8") as f:
    f.write(f"file '{os.path.abspath(intro)}'\n")
    f.write(f"file '{os.path.abspath(main)}'\n")
    f.write(f"file '{os.path.abspath(outro)}'\n")

# 步骤 2：调用 ffmpeg 合并
command = [
    "ffmpeg",
    "-f", "concat",
    "-safe", "0",
    "-i", "file_list.txt",
    "-c", "copy",
    output
]

try:
    subprocess.run(command, check=True)
    print("视频合并成功！输出文件:", output)
except subprocess.CalledProcessError as e:
    print("合并失败:", e)

# 清理临时文件
os.remove("file_list.txt")


视频合并成功！输出文件: final_output.mp4


In [55]:
import subprocess

input_video = "final_output.mp4"
output_video = "final_with_subs.mp4"
subtitle_file = "subtitles.srt"



filter_complex = """
[0:v]split[base][blur_source];
[blur_source]crop=in_w:in_h*0.3:0:in_h*0.7,boxblur=10[blurred_bar];
[base][blurred_bar]overlay=0:H-h[subbed];
[subbed]subtitles=subtitles.srt
"""

command = [
    "ffmpeg",
    "-i", input_video,
    "-filter_complex", filter_complex,
    "-c:a", "copy",  # 保留原音频
    "-y", output_video
]

subprocess.run(command, check=True)
print("✅ 成功：字幕已添加，且背景优化提升了可读性。输出文件:", output_video)


✅ 成功：字幕已添加，且背景优化提升了可读性。输出文件: final_with_subs.mp4


In [56]:
import subprocess

input_video = "final_output.mp4"
subtitle_file = "subtitles.srt"
output_video = "final_with_subs.mp4"

# 设置 filter_complex：在底部加 30% 高度的黑色半透明条，再加字幕
filter_complex = """
[0:v]drawbox=x=0:y=ih*0.7:w=iw:h=ih*0.3:color=black@0.5:t=fill[subbed];
[subbed]subtitles=subtitles.srt
"""

# 构造 ffmpeg 命令
command = [
    "ffmpeg",
    "-i", input_video,
    "-filter_complex", filter_complex,
    "-c:a", "copy",  # 保留原音频
    "-y",
    output_video
]

# 执行
subprocess.run(command, check=True)
print("✅ 成功：输出文件已添加黑色字幕背景条和字幕 →", output_video)


✅ 成功：输出文件已添加黑色字幕背景条和字幕 → final_with_subs.mp4
